In [2]:
import pandas as pd
import yfinance as yf
import requests
import time
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)
db = client.db

In [4]:
threaddf = pd.DataFrame.from_records(db.threads.find({'created_utc': {'$exists': 'true'}}))

In [6]:
threaddf['comments'] = 0
threaddf['comments'] = threaddf['comments'].astype('object')

In [7]:
def getData(datatype, **kwargs):
    url = f'https://api.pushshift.io/reddit/search/{datatype}/'
    payload = kwargs
    req = requests.get(url, params = payload)
    return req.json()['data']

def getIds(id):
    url = f'https://api.pushshift.io/reddit/submission/comment_ids/{id}'
    req = requests.get(url)
    return req.json()['data']

In [21]:
'''for i in range(356, len(threaddf)):
    print(i)
    threaddf.at[i,'comments'] = getIds(threaddf.loc[i,'id'])
    time.sleep(1)'''

356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590


In [28]:
for i in range(len(threaddf)):
    threaddf.at[i, 'numComments'] = len(threaddf.loc[i, 'comments'])

In [33]:
'''bulk_update = db.threads.initialize_unordered_bulk_op()
for index, row in threaddf.iterrows():
    _id = row['_id']
    commentArray = row['comments']
    commentAmount = row['numComments']
    bulk_update.find({'_id': _id}).update_one({'$set':{'comments': commentArray, 'numComments': commentAmount}})
bulk_update.execute()'''


"bulk_update = db.threads.initialize_unordered_bulk_op()\nfor index, row in threaddf.iterrows():\n    _id = row['_id']\n    commentArray = row['comments']\n    commentAmount = row['numComments']\n    bulk_update.find({'_id': _id}).update_one({'$set':{'comments': commentArray, 'numComments': commentAmount}})\nbulk_update.execute()"

In [ ]:
threaddf['comments']

In [5]:
'''subreddit = 'wallstreetbets'
lastoctober = 1569888000
thisoctober = 1601510400
size = 500'''

In [7]:
'''
df = pd.DataFrame()
after = lastoctober
before = thisoctober
while before > after:
    testdict = getData(datatype = 'submission', subreddit = subreddit, after = after, q = 'Daily Discussion Thread', before = before, size = size)
    if not testdict:
        break
    tempdf = pd.DataFrame.from_records(testdict)
    df = df.append(tempdf, ignore_index = True)
    after = df['created_utc'][len(df)-1]
    print(after)
    time.sleep(1)'''

1575630325
1581368695
1586370524
1591733184
1596829613
1601496415


In [7]:
import random
def pareComments(comments):
    random.shuffle(comments)
    return comments[0:int(len(comments)*0.3)]

In [8]:
threaddf['pareComments'] = threaddf['comments'].apply(pareComments)

In [9]:
for i in range(len(threaddf)):
    threaddf.at[i,'pareTotal'] = len(threaddf.loc[i, 'pareComments'])

In [13]:
import praw

In [14]:
secret = db.secret.find_one()['Secret']
reddit = praw.Reddit(client_id='-wpcPIbA7bhlpw', client_secret=secret, user_agent='sentiment')

In [100]:
commentdf = pd.DataFrame(columns = ['id', 'created_utc', 'body', 'score'])

In [101]:
for i in range(591, len(threaddf)):
    print(i)
    commentlist = threaddf.loc[i,'pareComments']
    commentlist = ['t1_' + x for x in commentlist]
    total = threaddf.loc[i, 'pareTotal']
    counter = 0
    for comment in reddit.info(commentlist):
        commentdf = commentdf.append({'id': comment.id, 'created_utc': comment.created_utc, 'body': comment.body, 'score': comment.score}, ignore_index = True)
        if total < 10:
            continue
        elif counter % (int(total / 10)) == 0:
            print('\t' + str(counter) + '/' + str(total))
        counter += 1

580
	0/5091.0
	509/5091.0
	1018/5091.0
	1527/5091.0
	2036/5091.0
	2545/5091.0
	3054/5091.0
	3563/5091.0
	4072/5091.0
	4581/5091.0
	5090/5091.0
581
582
583
584
	0/3469.0
	346/3469.0
	692/3469.0
	1038/3469.0
	1384/3469.0
	1730/3469.0
	2076/3469.0
	2422/3469.0
	2768/3469.0
	3114/3469.0
	3460/3469.0
585
	0/4516.0
	451/4516.0
	902/4516.0
	1353/4516.0
	1804/4516.0
	2255/4516.0
	2706/4516.0
	3157/4516.0
	3608/4516.0
	4059/4516.0
	4510/4516.0
586
	0/4343.0
	434/4343.0
	868/4343.0
	1302/4343.0
	1736/4343.0
	2170/4343.0
	2604/4343.0
	3038/4343.0
	3472/4343.0
	3906/4343.0
	4340/4343.0
587
	0/4698.0
	469/4698.0
	938/4698.0
	1407/4698.0
	1876/4698.0
	2345/4698.0
	2814/4698.0
	3283/4698.0
	3752/4698.0
	4221/4698.0
	4690/4698.0
588
	0/7992.0
	799/7992.0
	1598/7992.0
	2397/7992.0
	3196/7992.0
	3995/7992.0
	4794/7992.0
	5593/7992.0
	6392/7992.0
	7191/7992.0
	7990/7992.0
589
	0/6104.0
	610/6104.0
	1220/6104.0
	1830/6104.0
	2440/6104.0
	3050/6104.0
	3660/6104.0
	4270/6104.0
	4880/6104.0
	5490/6104.0
	610

In [102]:
commentdf

,id,created_utc,body,score
0,g6j58z6,1.601040e+09,"good thing they are, I finally gave up and sol...",1
1,g6j8vgc,1.601042e+09,Need a 10 bagger to buy in to today and hold u...,3
2,g6ja55u,1.601042e+09,[deleted],1
3,g6k38im,1.601057e+09,Mostly chart disagrees with you.,2
4,g6keujs,1.601063e+09,monday will be green as usual,6
...,...,...,...,...
40015,g79oznl,1.601543e+09,Are you going to publish your research?,5
40016,g79bjxq,1.601530e+09,Ah fucking boomer mentality,7
40017,g77wunh,1.601500e+09,Yea for no reason at all,1
40018,g780x3y,1.601502e+09,what are you looking to do? Hold long and exit...,1


In [103]:
db.comments.insert_many(commentdf.to_dict('records'))

In [114]:
df = pd.DataFrame.from_records(db.comments.find({'created_utc': {'$exists': 'true'}}))

In [115]:
df

,_id,id,created_utc,body,score
0,5fac60f84f1d40506940af6e,f24h9o9,1.569952e+09,[deleted],3
1,5fac60f84f1d40506940af6f,f23y8fn,1.569941e+09,[**Donald J. Trump**@realDonaldTrump](https://...,7
2,5fac60f84f1d40506940af70,f24vyaa,1.569961e+09,I subscribe to this reality,3
3,5fac60f84f1d40506940af71,f24dxww,1.569950e+09,nice,1
4,5fac60f84f1d40506940af72,f243mu5,1.569944e+09,Na that’s just me who farted after eating chip...,1
...,...,...,...,...,...
2650289,5faf404fa6460ea208f39e70,g79oznl,1.601543e+09,Are you going to publish your research?,5
2650290,5faf404fa6460ea208f39e71,g79bjxq,1.601530e+09,Ah fucking boomer mentality,7
2650291,5faf404fa6460ea208f39e72,g77wunh,1.601500e+09,Yea for no reason at all,1
2650292,5faf404fa6460ea208f39e73,g780x3y,1.601502e+09,what are you looking to do? Hold long and exit...,1


In [116]:
duplicatedf = df.loc[df.duplicated(subset = 'id', keep = 'first')]

In [117]:
duplicatedf

,_id,id,created_utc,body,score
